In [3]:
# ребалансировка классов для несбалансированных классов (подробнее здесь https://www.kaggle.com/viktorandriichuk/vodafon-acclrtr-3rd-place-adaboost-randomforest/edit)
def resample(features, target, repeat_down, repeat_up):
    features_zeros = features[target == 0]
    features_ones = features[target == 1]
    features_two = features[target == 2]
    target_zeros = target[target == 0]
    target_ones = target[target == 1]
    target_two = target[target == 2]

    features_resampled = pd.concat([features_zeros.sample(n =(int(len(features_zeros)*repeat_down)), 
                                                            replace=False,random_state=2)] + [features_ones] 
                                   + [features_two.sample(n =(int(len(features_two)*repeat_up)), 
                                                            replace=True,random_state=2)])
    
    target_resampled = pd.concat([target_zeros.sample(n =(int(len(target_zeros)*repeat_down)),
                                                        replace=False)] + [target_ones] 
                                 + [target_two.sample(n =(int(len(target_two)*repeat_up)), 
                                                            replace=True)])
    features_resampled, target_resampled = shuffle(features_resampled, target_resampled, random_state=12345)
    features_resampled.reset_index(drop = True, inplace = True)
    target_resampled.reset_index(drop = True, inplace = True)
    return features_resampled, target_resampled

base_up = pd.DataFrame()

model_RFC = RandomForestClassifier(n_estimators = 50, random_state = 2)

for i in np.arange(0.37, 0.46, 0.02):
    for j in np.arange(2.2, 2.5, 0.05):
        features_resampled, target_resampled = resample(features_train_big, target_train_big, i , j)
        model_RFC.fit(features_resampled, target_resampled)
        base_up.loc[str(round(i,2)) + " " +str(round(j,2)) ,'Precision'] = precision_score(target_valid, model_RFC.predict(features_valid), average='macro')
        base_up.loc[str(round(i,2)) + " " +str(round(j,2)) ,'Recall'] = recall_score(target_valid, model_RFC.predict(features_valid), average='macro')
        base_up.loc[str(round(i,2)) + " " +str(round(j,2)),'F1'] = f1_score(target_valid, model_RFC.predict(features_valid), average='macro')

NameError: name 'pd' is not defined

In [2]:
#**Функция print_basic_info, для вывода информации о массиве, и его переменных.**

#* base - название базы данных
#* info - 1: вывод информации о массиве, другое: не вывод
#* describe - 1: вывод описания переменных массива, другое: не вывод        
#* duplicat - 1: вывод количества полных дублей
#* head - n: вывод примера базы (вывод n - строк), n < 1: не вывод

def print_basic_info(base, info, describe, duplicat, head):
    if info == 1:
        print(base.info())  
    if head >= 1:
        display(base.head(head))
    if describe == 1:
        for i in base.columns:
            print(base[i].describe())
    if duplicat == 1:
        print(base[base.duplicated() == True][base.columns[0]].count())

In [1]:
#__Функция ft_namecount__, для вывода названия переменной, частотной нормированной таблицы и описания переменной.

#5 входных параметров:

#* *base* - название базы данных
#* *name* - название переменной в базе
#* *table* - 1: вывод частотной нормированной таблицы, 0: не вывод
#* *sort* - 1: сортировка таблицы по лейблам переменной, 0: не сортировка
#* *describe* - 1: вывод описания переменной, 0: не вывод
#* *hist* - 1: вывод описания переменной, 0: не вывод
#* *bins* -  кол-во столцов для hist, по-умолчанию auto

def ft_name_count (base, name , table, sort, describe, hist, bins='auto'):
    if table != 0:
        s = (base[name].value_counts(normalize=True))
        if sort != 0:
            s.sort_index(inplace=True)
        print(s)
    if describe != 0:
        print(base[name].describe())
    
    if hist != 0:
        base[name].hist(bins=bins)

In [1]:
# Беглый анализ колонки и удаление, если много пропусков
col_for_drop = []

def col_info_train(col):

    if col.isnull().sum() > 0:
        percent_isnull = (col.isnull().sum() / len(col)) * 100
        print(col.name)
        print('Количество пропусков: {}'.format(col.isnull().sum()))
        print('% пропусков: {}'.format(percent_isnull))
        print('---')       
        
    
        # Удалю признак, в котором есть пропусков больше 10%
        if percent_isnull > 10:
            col_for_drop.append(col.name)
            train.drop([col.name], axis='columns', inplace=True)
        

for col in train:
    col_info_train(train[col]) 

In [ ]:
# Просмотр гистограммы для признака
def feature_hist(data, col):
    sns.countplot(data[ col])
    plt.title('Histogram for default')

In [1]:
# Беглый просмотр данных в признаке
def col_info(data, col, bins):
    percent_isnull = (data[col].isnull().sum()) / len(data[col]) * 100
    print('Название колонки: {}'.format(data[col].name))
    print('Количество пропусков: {}'.format(data[col].isnull().sum()))
    print('% пропусков: {}'.format(round(percent_isnull, 2)))
    print('{},'.format(data[col].describe()))
    print('Распределение:\n{},'.format(data[col].value_counts()))
    data[col].hist(bins=bins)

In [ ]:
# Визуализация признаков и пропусков
def viz_na(data):
    """NA visualisation"""
    global cols
    cols = data.columns # запишем названия строки сделаем переменную глобальной
    # определяем цвета 
    # желтый - пропущенные данные, синий - не пропущенные
    colours = ['#000099', '#ffff00'] 
    sns.heatmap(data[cols].isnull(), cmap=sns.color_palette(colours))
    plt.show()


viz_na(train)
viz_na(test)

In [ ]:
# Статистика пропусков 
def stat_na_per_percent(data):
    print(f'{data.shape}')
    for col in data.columns:
        pct_missing = np.mean(data[col].isnull())
        if pct_missing > 0: # выведем только те, где больше 0
            print('{} - {}%'.format(col, round(pct_missing*100)))
    print("END", end = '\n\n')
    
stat_na_per_percent(train)
stat_na_per_percent(test)

In [ ]:
# Пропущенные данные и выбросы

# В добавок к некорректному определению типов данных, другая частая проблема — это пропуски в данных.
# У наличия пропусков могут быть разные причины, но пропуски нужно либо заполнить, либо исключить из набора полностью.

# Для начала попробуем оценить масштаб проблемы

# Функция считает ошибочные данные в колонке
def missing_values_table(df):
        # Всего ошибочных данных (это я уже сделал выше, но выведу их и здесь для наглядности)
        mis_val = df.isnull().sum()
        
        # Процент ошибочных данных
        mis_val_percent = 100 * df.isnull().sum() / len(df)
        
        # Тип данных каждой колонки
        sm_cols = [c for c in sm.columns]

        cat_dtypes_dict = {}

        for ct in sm_cols:
            cat_dtypes_dict.update({sm[ct].name: sm[ct].dtype})

        cat_dtypes = pd.Series(cat_dtypes_dict) 
      
        # Сделаем датафрейм с результатами
        mis_val_table = pd.concat([mis_val, mis_val_percent, cat_dtypes], axis=1)
        
        # Переназовем колонки
        mis_val_table_ren_columns = mis_val_table.rename(
        columns = {0 : 'Missing Values', 1 : '% of Total Values', 2 : 'Types'})
        
#         # Отсортируем данные и исключим колонки, в которых нет пропусков
#         mis_val_table_ren_columns = mis_val_table_ren_columns[
#             mis_val_table_ren_columns.iloc[:,1] != 0].sort_values(
#         '% of Total Values', ascending=False).round(1)

        # Отсортируем данные с учетом колонок, в которых нет пропусков
        mis_val_table_ren_columns = mis_val_table_ren_columns.sort_values(
        '% of Total Values', ascending=False).round(1)
        
        # Выведем итоговую информацию
        print ("Your selected dataframe has " + str(df.shape[1]) + " columns.\n"      
            "There are " + str(mis_val_table_ren_columns.shape[0]) +
              " columns that have missing values.")
        
        # Вернем датафрейм с информацией об ошибочных данных
        return mis_val_table_ren_columns
    
missing_values_table(sm)

In [ ]:
# Выделим числовые и категориальные признаки в отдельные листы:

categorical_columns = [c for c in sm.columns if sm[c].dtype.name == 'object']
numerical_columns   = [c for c in sm.columns if sm[c].dtype.name != 'object']
print('Категориальные признаки:', categorical_columns)
print('Числовые признаки:', numerical_columns)
                       